### Imports

In [1]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 169 µs (started: 2023-06-02 14:34:38 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

dir = os.path.dirname(os.getcwd())
print(dir)

method = params["method"][params["method_idx"]]
print("Method:\t\t", method)

""" group = params["group"][params["group_idx"]]
print("Group:\t\t", group)

subgroups = [str(k + 1) for k in range(group[1])]
print("Subgroup:\t", subgroups) """

dimensions = params["dimensions"]
dimension = dimensions[0]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

/home/ealvarez/Project/GNN_Filter
Method:		 dgi
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 3.15 ms (started: 2023-06-02 14:34:38 -05:00)


### Edge embeddings

In [3]:
# get edges embeddings

edge_embeddings_global(method, dimension, groups_id, subgroups_id)

100%|██████████| 3/3 [52:08<00:00, 1042.75s/it]

time: 52min 8s (started: 2023-06-02 13:19:56 -05:00)


In [4]:
df_edge_embeddings = pd.read_csv("output/edge_embeddings/edge-embeddings_{}_{}_{}_{}_{}.csv".format(groups_id[0], subgroups_id[groups_id[0]][0], method, dimension, "L2"), index_col=[0, 1])
df_edge_embeddings.head()

0             1             2
0 1  1.143202e-03  7.768660e-09  1.940209e-03
  2  5.718989e-09  1.909690e-11  1.120634e-08
  3  1.714288e-03  7.063369e-08  2.939478e-03
  4  1.809654e-04  1.093625e-07  3.256343e-04
  5  3.382710e-04  5.913610e-11  5.694203e-04

time: 517 ms (started: 2023-06-02 14:16:09 -05:00)


### Concat edge embeddings

In [7]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    for subgroup in tqdm(subgroups_id[group]):
        df_edge_embeddings = pd.read_csv("output/edge_embeddings/edge-embeddings_{}_{}_{}_{}_{}.csv".format(group, subgroup, method, dimension, "L2"), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [subgroup] * len(df_edge_embeddings)

        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
    df_edge_embeddings_concat.to_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index=True)

100%|██████████| 3/3 [03:12<00:00, 64.17s/it]

time: 3min 12s (started: 2023-06-02 14:18:02 -05:00)


In [13]:
df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(groups_id[0], method, dimension, "L2"), index_col=[0, 1])
df_edge_embeddings_concat.head()

0             1             2  subgroup
0 1  1.143202e-03  7.768660e-09  1.940209e-03         1
  2  5.718989e-09  1.909690e-11  1.120634e-08         1
  3  1.714288e-03  7.063369e-08  2.939478e-03         1
  4  1.809654e-04  1.093625e-07  3.256343e-04         1
  5  3.382710e-04  5.913610e-11  5.694203e-04         1

time: 4.74 s (started: 2023-06-02 14:53:34 -05:00)


In [9]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/plot/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.png".format(group, method, dimension, "L2"))
    # plt.show()
    plt.close()

  0%|          | 0/3 [00:00<?, ?it/s]

### Outliers detection

In [21]:
# Outlier detection (HDBSCAN)
""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

time: 9min 48s (started: 2023-06-02 11:04:08 -05:00)


In [60]:
# outlier detection (ECOD)
dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|█████████████████████████████| 3/3 [00:57<00:00, 19.19s/it]

time: 58.2 s (started: 2023-06-02 16:06:33 -05:00)


In [5]:
# plot outliers/inliers
for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/plot/edge_embeddings/edge-embeddings_outlier_{}_{}_{}_{}.png".format(group, method, dimension, "L2"))
    # plt.show()
    plt.close()

3it [13:34, 271.43s/it]

time: 13min 34s (started: 2023-06-02 11:56:46 -05:00)


###  Filter common edges

In [64]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id[group]:
        df_nodes = pd.read_csv("output/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(group, subgroup))
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []

    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[str(row[-1])]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
    df_edge_embeddings_concat_filter

11008782it [05:14, 35044.58it/s]          | 0/3 [00:00<?, ?it/s]
15378760it [07:07, 35968.25it/s] | 1/3 [05:26<10:52, 326.18s/it]
5439027it [02:31, 35984.36it/s]  | 2/3 [12:45<06:32, 392.66s/it]
100%|████████████████████████████| 3/3 [15:21<00:00, 307.27s/it]

time: 15min 21s (started: 2023-06-02 16:08:00 -05:00)


In [99]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

,level_0,level_1,0,1,2,subgroup
0,59.0049,369.1213,0.001714,7.063369e-08,0.002939,1
1,59.0049,558.4273,0.003690,3.074061e-08,0.006267,1
2,59.0049,641.1223,0.006839,5.105531e-07,0.011797,1
3,59.0049,650.2145,0.005214,6.020984e-07,0.009044,1
4,59.0049,977.8237,0.018821,7.463387e-06,0.033454,1


time: 13.1 ms (started: 2023-06-02 17:16:22 -05:00)


In [100]:
# count edges and filter by count
dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # count
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["level_0", "level_1"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]

    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

100%|█████████████████████████████| 3/3 [00:08<00:00,  2.90s/it]

time: 8.71 s (started: 2023-06-02 17:16:29 -05:00)


In [107]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,716.5216,746.4967
1,341.1770,602.1407
2,379.0830,463.1124
3,267.0740,571.1777
4,688.4907,814.4931


time: 9.43 ms (started: 2023-06-02 17:36:56 -05:00)


In [123]:
# get weight by subgroups
dict_df_edges_filter_weight = {}

for group in tqdm(groups_id):
    df_edges_filter_weight = dict_df_edges_filter[group].copy()

    s = []
    t = []
    for row in df_edges_filter_weight.itertuples():
        if row[1] > row[2]:
            s.append(row[2])
            t.append(row[1])
        else:
            s.append(row[1])
            t.append(row[2])
    df_edges_filter_weight["source"] = s
    df_edges_filter_weight["target"] = t

    df_edges_filter_weight.sort_values(["source", "target"], ascending=True, inplace=True)
    df_edges_filter_weight["idx"] = df_edges_filter_weight["source"].astype(str) + "-" + df_edges_filter_weight["target"].astype(str)
    list_aux = df_edges_filter_weight.iloc[:, -1].values

    for subgroup in tqdm(subgroups_id[group]):
        df_edges = pd.read_csv("output/preprocessing/edges/edges_{}_{}.csv".format(group, subgroup))
        df_edges.sort_values(["source", "target"], ascending=True, inplace=True)
        df_edges["idx"] = df_edges["source"].astype(str) + "-" + df_edges["target"].astype(str)
        
        filter = df_edges["idx"].isin(list_aux)
        temp = df_edges[filter]
        list_temp = temp.iloc[:, -2].values
        df_edges_filter_weight["subgroup{}".format(subgroup)] = list_temp
        
    df_edges_filter_weight.drop(["idx"], inplace=True, axis=1)
    
    dict_df_edges_filter_weight[group] = df_edges_filter_weight


100%|██████████████████████████| 3/3 [00:49<00:00, 16.59s/it]

time: 49.8 s (started: 2023-06-02 17:55:53 -05:00)


In [124]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
17925,59.0137,349.2758,0.796859,0.788310,0.560861,0.551982,0.807207
17920,59.0137,350.2793,0.807013,0.779306,0.576143,0.563560,0.808223
17919,59.0137,350.3033,0.854806,0.751778,0.539172,0.650983,0.883973
17773,59.0137,389.2739,0.823682,0.886260,0.723124,0.515230,0.917952
17772,59.0137,389.2907,0.887545,0.930750,0.839795,0.800743,0.914504


time: 13.3 ms (started: 2023-06-02 17:56:47 -05:00)


### Filter by STD and average weight

In [135]:
# dispersion (std)

# dict_df_common_edges = {}

for group in tqdm(groups_id):
    df_edges_filter_weight = dict_df_edges_filter_weight[group]

    # calculate std
    df_edges_filter_weight["std"] = np.std(df_edges_filter_weight.iloc[:, 2:], axis=1)

    # filter std < 0.3
    df_edges_filter_weight_std = df_edges_filter_weight[df_edges_filter_weight["std"] < 0.3]

    # average weight
    df_edges_filter_weight_std_avg = df_edges_filter_weight_std.iloc[:, :-1]
    df_edges_filter_weight_std_avg["weight"] = df_edges_filter_weight_std_avg.iloc[:, 2:].mean(axis=1)
    df_edges_filter_weight_std_avg = df_edges_filter_weight_std_avg.iloc[:, [0, 1, -1]]
    df_edges_filter_weight_std_avg.reset_index(drop=True, inplace=True)

    # save
    df_edges_filter_weight_std_avg.to_csv("output/common_edges/common_edges_{}_{}.csv".format(group, method, dimension, "L2"), index=False)

    # dict_df_common_edges[group] = df_edges_filter_weight_std_avg

100%|██████████████████| 3/3 [00:02<00:00,  1.20it/s]

time: 2.5 s (started: 2023-06-02 18:16:03 -05:00)


In [137]:
df_common_edges = pd.read_csv("output/common_edges/common_edges_{}_{}.csv".format(groups_id[0], method, dimension, "L2"))
df_common_edges.head()

,source,target,weight
0,59.0137,349.2758,0.701044
1,59.0137,350.2793,0.706849
2,59.0137,350.3033,0.736142
3,59.0137,389.2739,0.773249
4,59.0137,389.2907,0.874667


time: 21.5 ms (started: 2023-06-02 18:20:16 -05:00)


In [139]:
# show details
for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/common_edges/common_edges_{}_{}.csv".format(groups_id[0], method, dimension, "L2"))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

100%|██████████████████████████| 3/3 [00:00<00:00, 20.60it/s]

Group: WT
Num. nodes: 1913
Num. edges: 22986

Group: zwf1^
Num. nodes: 1913
Num. edges: 22986

Group: pck1^
Num. nodes: 1913
Num. edges: 22986

time: 151 ms (started: 2023-06-02 18:21:25 -05:00)
